In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
df = pd.read_csv("uci-secom.csv")
df.shape

(1567, 592)

In [3]:
df.head(2)

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.006,208.2045,-1


In [4]:
df["Pass/Fail"].value_counts()

Pass/Fail
-1    1463
 1     104
Name: count, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [6]:
df.isnull().sum().describe()

count     592.000000
mean       70.863176
std       241.476304
min         0.000000
25%         2.000000
50%         6.000000
75%         9.000000
max      1429.000000
dtype: float64

In [7]:
df.select_dtypes(include="O").head(1)

,Time
0,2008-07-19 11:55:00


In [8]:
lable_name = "Pass/Fail"
X, y = df.drop(columns=[lable_name, "Time"]), df[lable_name]
X.shape, y.shape

((1567, 590), (1567,))

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [10]:
cat_col = X.select_dtypes(include="object").columns
print(cat_col)
X_train[cat_col] = X_train[cat_col].astype("category")
X_test[cat_col] = X_test[cat_col].astype("category")

Index([], dtype='object')


* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html
* https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting
* https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_categorical.html

In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

ct = make_column_transformer(
      (StandardScaler(),
       make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"),
       make_column_selector(dtype_include="category")))

In [12]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import make_pipeline

hist_pipe = make_pipeline(ct, HistGradientBoostingClassifier(random_state=42))
hist_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x134356170>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x134a18d30>)])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(random_state=42))])

In [13]:
from sklearn.model_selection import cross_validate

scoring = "neg_mean_absolute_percentage_error"
n_cv_folds = 3

cv_result = cross_validate(hist_pipe, X_test, y_test, cv=n_cv_folds, scoring=scoring)
cv_result

{'fit_time': array([0.37132001, 0.34408021, 0.35995507]),
 'score_time': array([0.00850821, 0.00801492, 0.01072311]),
 'test_score': array([-0.11320755, -0.11538462, -0.11538462])}

In [14]:
# fit & predict
hist_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x134356170>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x134a18d30>)])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(random_state=42))])

In [15]:
# ct.get_feature_names_out()

https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance

In [16]:
from sklearn.inspection import permutation_importance

model = hist_pipe
result = permutation_importance(model, X_train, y_train,
                                n_repeats=3, random_state=0)

In [17]:
feature_names = ct.get_feature_names_out()

In [18]:
# for i in result.importances_mean.argsort()[::-1]:
#     if result.importances_mean[i] - 2 * result.importances_std[i] > 0:
#         print(f"{feature_names[i]:<8}"
#               f"{result.importances_mean[i]:.3f}"
#               f" +/- {result.importances_std[i]:.3f}")

In [19]:
hgbr_score = hist_pipe.score(X_test, y_test)
hgbr_score

0.9490445859872612

In [21]:
y_pred_hgbr = hist_pipe.predict(X_test)
y_pred_hgbr[:5]

array([-1, -1, -1, -1, -1])